# ETEL

In [ ]:
import numpy as np
import torch

dtype = torch.float32
dtype_np = np.float32
torch.set_default_dtype(dtype)

def simulator(T, 𝜇, 𝜎=1.0):
    N = np.size(𝜇)
    x = 𝜎 * np.random.normal(size=(N,T)) + np.atleast_2d(𝜇).T
    return x 

N, T = 100, 10
𝜇s = - 0.5 * np.ones(N) # np.random.normal(size=N)
X = simulator(T, 𝜇=𝜇s)
X = torch.tensor(X, dtype=dtype).unsqueeze(-1)
X.shape

In [ ]:
import scipy.optimize as spo
from scipy.special import logsumexp

class EE(torch.nn.Module):
    def __init__(self, D, M, K):
        super().__init__()
        self.D = D # dimensionality of each data point x_i, i=1,...,T
        self.M = M # dimensionality of modle parameter 𝜃
        self.K = K # number of estimating equations
        self.layer1 = torch.nn.Linear(D+M, K)
    def forward(self, x, 𝜃):
        x_full = torch.cat((x,𝜃.unsqueeze(-2).expand(-1,X.shape[-2],-1)),dim=-1)
        return self.layer1(x_full)

def Φ(𝜆, G):
    𝜂 = torch.matmul(G, 𝜆.unsqueeze(-1)).squeeze(-1)
    return torch.logsumexp(𝜂, axis=-1)

def Φ_np(𝜆, G):
    𝜂 = G.dot(𝜆)    
    return logsumexp(𝜂, axis=-1)

def gradΦ(𝜆, G):
    w = torch.exp(log_w_opt(𝜆, G))
    return G.dot(w)

def solve_𝜆(G):
    with torch.no_grad():
        N, K = G.shape[0], G.shape[-1]
        𝜆0 = np.zeros((N, K))
        𝜆 = np.zeros_like(𝜆0)
        for i in range(N):
            def Φ_G(𝜆):
                return Φ_np(𝜆, G[i].numpy())
            𝜆[i] = spo.minimize(Φ_G, 𝜆0[i])['x']
    return torch.tensor(𝜆,dtype=dtype)

def log_w_opt(𝜆, G):
    𝜂 = torch.matmul(G, 𝜆.unsqueeze(-1)).squeeze(-1)
    return 𝜂 - torch.logsumexp(𝜂,axis=-1).unsqueeze(-1)

def log_pX𝜃(g, X, 𝜃):
    G = g(X, 𝜃)
    𝜆 = solve_𝜆(G)
    log_w = log_w_opt(𝜆, G)
    return log_w.sum(axis=-1)


In [ ]:

D, M = 1, 1
K = M # use as many estimating equations as there are model parameters

g = EE(D,M,K)

# manually set estimating equations to g(x,𝜃) = x - 𝜃 such that the model fits the mean 𝜃 = E[X]
d = g.layer1.state_dict()
d['weight'] = torch.tensor([[1.,-1.]])
d['bias'] = torch.tensor([0.])
g.layer1.load_state_dict(d)


𝜃 = torch.linspace(-2, 1, N).unsqueeze(-1) # range of test values for 𝜃
X = X[0].unsqueeze(0).repeat(N,1,1) # fix one dataset


In [ ]:
import matplotlib.pyplot as plt


ll = log_pX𝜃(g, X, 𝜃)
# exponentially tilted empirical likelihood 
plt.plot(𝜃.detach().numpy(), ll.detach().numpy() - ll.min().detach().numpy(), '.')
# compare to true likelihood
plt.plot(𝜃.detach().numpy(), (- 0.5 * (𝜃.squeeze(1) - X.mean(axis=(1,2)))**2).detach().numpy()) 